In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.013798713684082031
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 49.41994030033922
200000 72.87164316177231
300000 87.99801495800078
400000 86.89081702842446
500000 77.90584197682708
600000 72.60055242594491
700000 75.83612504326231
800000 70.68123225271937
900000 74.15695834629759
1000000 76.6536489270464
1100000 76.49381661336507
1200000 65.33256548674369
1300000 25.01273739387014
1400000 70.27981068751465
1500000 76.38198011578721
1600000 70.56551360705446
1700000 70.46121609996352
1800000 70.55356745669187
1900000 75.88801187029874
2000000 74.2497427422723
2100000 69.9469234227887
2200000 73.65842106521903
2300000 71.39973148779794
2400000 72.98778317335636
2500000 21.926183069416183
2600000 68.87330824018498
2700000 91.61095576214942
2800000 91.38718986603342
2900000 94.59614986846714
3000000 92.91003052171631
31

30100000 79.09462221227349
30200000 74.6261866897912
30300000 75.78624242888063
30400000 75.89811594224437
30500000 72.29560700963809
30600000 76.18521437917235
30700000 71.89492822172875
30800000 15.767488456140354
30900000 76.03565936790439
31000000 70.41590910686446
31100000 74.54970282745876
31200000 71.86769087722166
31300000 77.90395090396848
31400000 74.12220326564406
31500000 78.91402873856586
31600000 80.42515182438929
31700000 77.98556190950448
31800000 26.21694516109605
31900000 81.35437641469113
32000000 75.26172735099225
32100000 80.6010618107081
32200000 69.26835938152344
32300000 33.19372244392394
32400000 75.48123502297521
32500000 75.48552793641751
32600000 75.53848724070716
32700000 74.29961033012813
32800000 73.73383267933693
32900000 70.48485496548294
33000000 72.79430179699419
33100000 71.57012022309344
33200000 52.00930605601356
33300000 37.539191226164654
33400000 65.93687009220928
33500000 71.99390473642941
33600000 71.93862476087155
33700000 72.6627672966675
33

60500000 64.88501011912093
60600000 68.05471451288456
60700000 64.71960613158569
60800000 69.14062509947365
60900000 66.89399702937061
61000000 71.19459590018585
61100000 29.006622943707683
61200000 75.76691729225873
61300000 71.87725752551589
61400000 72.46377320025826
61500000 72.9577214032268
61600000 72.15459273130719
61700000 72.13150928540297
61800000 71.84475614724066
61900000 73.34006416936673
62000000 35.40214643335376
62100000 61.221399834292306
62200000 72.26844728393505
62300000 68.77781913185137
62400000 67.54487211595817
62500000 65.6867078124929
62600000 63.774938331277056
62700000 62.38273732516815
62800000 18.17212210543054
62900000 66.68379791316318
63000000 63.22018349884346
63100000 66.35310464348711
63200000 65.41723609564242
63300000 69.33129007852237
63400000 68.39406074712107
63500000 49.32035929902102
63600000 51.70411132866878
63700000 75.99917661351587
63800000 70.52389049181771
63900000 72.61447294060862
64000000 71.45380810305515
64100000 76.74230257700103


90800000 72.3939865609688
90900000 66.24719573644425
91000000 69.71674566688692
91100000 71.30354985951905
91200000 66.12741651969365
91300000 72.46209361585792
91400000 69.16207635919064
91500000 67.20608199522906
91600000 69.86967804795647
91700000 50.94519612761721
91800000 36.57463738808944
91900000 60.424226965854224
92000000 63.12269467364388
92100000 67.55360842197365
92200000 61.476531945312935
92300000 19.34876669770045
92400000 60.348647611051284
92500000 64.52754869815253
92600000 58.54755846233921
92700000 62.4619543731892
92800000 48.03051071209358
92900000 43.21027669749827
93000000 64.00932649450772
93100000 66.25041609827984
93200000 67.26026769180504
93300000 69.18091084882084
93400000 63.538837813106326
93500000 69.46382225946694
93600000 66.0120163719507
93700000 66.41016120525761
93800000 69.51388851585534
93900000 67.13598667025965
94000000 63.911740928656755
94100000 65.30206199724286
94200000 41.53556816627896
94300000 43.08811522926198
94400000 62.44429143743234

120300000 66.09440490267265
120400000 73.09459174899435
120500000 72.39372369769609
120600000 68.71551150943809
120700000 67.03190234424144
120800000 66.61481615759145
120900000 62.47876212582674
121000000 61.090791896761715
121100000 59.004272594406615
121200000 63.70062896928594
121300000 62.52853904048162
121400000 63.638983770780825
121500000 22.230688933949587
121600000 56.92355013624958
121700000 58.33175683884139
121800000 31.911380308540203
121900000 45.575696344650574
122000000 58.971249175304074
122100000 46.642002021493276
122200000 38.81476106148257
122300000 61.76163129743371
122400000 59.70622816510701
122500000 60.04102510953956
122600000 60.907213212181816
122700000 62.6838028460836
122800000 67.1626505657523
122900000 66.58962582971256
123000000 66.48531078280408
123100000 65.64124418354929
123200000 65.4002576574831
123300000 70.18851435274131
123400000 64.99642279081988
123500000 60.29016608706407
123600000 63.42604931731706
123700000 58.673658061315464
123800000 55.

...

638800000 1.9306750561132533
638900000 2.1596829686671146
639000000 2.11697129475465
639100000 2.0655674272629083
639200000 1.9289857350276256
639300000 2.208004592985659
639400000 2.4543950696792205
639500000 2.2084320836141025
639600000 1.9288671111222433
best so far: 0
type: [1, 1, 3, 5, 9] 135
cases of this type: 7381125
639700000 2.013089023327402
639800000 1.841795735081367
639900000 1.941677385630958
640000000 1.786449453599751
640100000 1.8669630197640459
640200000 1.9599548038587467
640300000 1.9295018955707768
640400000 1.8198207875030668
640500000 1.7751715639561694
640600000 1.8002815942463122
640700000 1.7805371911160186
640800000 1.774301983311158
640900000 1.8743092058336257
641000000 2.0775360153816953
641100000 1.923776043695009
641200000 1.7727474753274244
641300000 1.717270889063992
641400000 1.7035581649542093
641500000 1.6061052858946603
641600000 1.7152853719788466
641700000 1.7402423026314187
641800000 1.9210573428523599
641900000 1.9883276228952216
642000000 1.8

666300000 2.6662024823951898
666400000 2.7909291731856327
666500000 2.340782797346931
666600000 2.0708962311776307
666700000 1.9309083968847216
666800000 2.710582572406176
666900000 2.6208621444987386
667000000 2.4285025195165217
667100000 1.8684820831971183
667200000 1.6224413925628227
667300000 2.383501232983516
667400000 2.399456561120093
667500000 2.39796467058022
667600000 2.540066840329659
667700000 2.2687539575697824
best so far: 0
type: [1, 1, 5, 3, 9] 135
cases of this type: 4428675
667800000 1.8612057873834738
667900000 1.2401286839424253
668000000 0.9846575426362502
668100000 1.1778142178328217
668200000 1.3626895380234458
668300000 1.3554565334831163
668400000 1.1049096535762581
668500000 1.3149832124478082
668600000 1.3511852085652352
668700000 0.9683294126574874
668800000 0.8347254762944878
668900000 1.3751176740397353
669000000 1.2727862609012246
669100000 1.1958245793535969
669200000 1.3630901955742691
669300000 1.5204937749348097
669400000 1.5955703750496606
669500000 

690800000 1.9001532046676748
690900000 0.9154878640788595
691000000 2.024274501825889
691100000 1.491106109116378
691200000 0.49904274614590804
691300000 1.5331785009205936
691400000 1.5390160483568216
691500000 0.9963887551112746
691600000 1.5426335250311076
691700000 0.5961707907100293
691800000 0.9375334568567713
691900000 0.9839351224547207
692000000 0.7787789637888856
692100000 0.9673406459443821
692200000 1.1958202052969098
692300000 0.9858078123718493
692400000 1.4905624212542383
692500000 0.8302003870697439
692600000 1.073745025578585
692700000 1.7985393471098523
692800000 1.1998204153513432
692900000 1.425159086680457
693000000 0.9532088266754746
693100000 0.9519691623067259
693200000 0.9677930451589147
693300000 1.3782255124703413
693400000 0.9562520450561841
693500000 1.2734638746874622
693600000 1.085645052095492
693700000 0.6886519682969392
693800000 1.2368824752719965
693900000 0.9095978506092813
694000000 0.9847039443805098
694100000 0.9618813188067502
694200000 0.794040

712600000 0.13011277927657167
712700000 0.118817946437441
712800000 0.05330142877035009
712900000 0.23124844635949732
best so far: 0
type: [1, 9, 1, 3, 5] 135
cases of this type: 151875
713000000 0.12000914742145803
713100000 0.0580190795052555
best so far: 0
type: [1, 9, 1, 5, 3] 135
cases of this type: 91125
713200000 0.05752235596484939
best so far: 0
type: [1, 9, 1, 15, 1] 135
cases of this type: 30375
best so far: 0
type: [1, 9, 3, 1, 5] 135
cases of this type: 50625
best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.05211720474181837
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.05864449156788786
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type

best so far: 0
type: [5, 9, 1, 3, 1] 135
cases of this type: 243
best so far: 0
type: [5, 9, 3, 1, 1] 135
cases of this type: 81
best so far: 0
type: [5, 27, 1, 1, 1] 135
cases of this type: 27
best so far: 0
type: [9, 1, 1, 1, 15] 135
cases of this type: 50625
best so far: 0
type: [9, 1, 1, 3, 5] 135
cases of this type: 16875
best so far: 0
type: [9, 1, 1, 5, 3] 135
cases of this type: 10125
best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.00023529819286266962
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cas